In [161]:
# url of website
from pymongo import MongoClient
import requests
from bs4 import BeautifulSoup
import pandas as pd
URL = 'https://www.worldometers.info/coronavirus/'

# get the page
page = requests.get(url=URL)

# soup
soup = BeautifulSoup(page.text)

In [162]:
today = 0   # Table pointer

In [163]:
if today == 1:
    # today's table
    full_table = soup.find('table', {'id':'main_table_countries_today'})
else:
    # sometimes today's table are not updated yet and contain many blanks, then use yesterday's table
    full_table = soup.find('table', {'id':'main_table_countries_yesterday'})

In [164]:
def generate_dataframe_world(html_table):
    
    # generate columns for the df

    country = [] # country list initialized
    confirmed = [] # confirmed list initialized
    new_case = [] # new case list initialized
    death = [] # death list initialized
    recover = [] # recover list initialized

    for row in html_table.find_all('tr'):   # itterate over html table rows 
        cell = row.find_all('td')

        if (len(cell) == 22):   
            country.append(cell[1].find(text = True))   # Cell entries appended to their respective lists
            confirmed.append(cell[2].find(text = True))
            new_case.append(cell[3].find(text = True))
            death.append(cell[4].find(text = True))
            recover.append(cell[6].find(text = True))

    # put together in to df

    df = pd.DataFrame()
    df['Country'] = country
    df['Confirmed'] = confirmed
    df['New_cases'] = new_case
    df['Deaths'] = death
    df['Recovered'] = recover

    # remove some unnecessary rows
    df.set_index('Country', inplace=True)
    df.drop(['\n','World','Total:'], inplace=True)
    df.reset_index(inplace=True)
    df.fillna('0', inplace=True)

    # remove symbols
    df['Confirmed'] = df.Confirmed.str.replace(',','')
    df['New_cases'] = df.New_cases.str.replace(',','')
    df['New_cases'] = df.New_cases.str.replace('+','')
    df['Deaths'] = df.Deaths.str.replace(',','')
    df['Recovered'] = df.Recovered.str.replace(',','')
    df['Recovered'] = df.Recovered.str.replace('N/A','0')
    df.replace(r'^\s*$', 0, regex=True, inplace=True) # replace blank spaces in cells with 0

    # set data type
    df = df.astype({'Country':str, 'Confirmed':int, 'New_cases':int, 'Deaths':int, 'Recovered':int})
    
    return df

In [165]:
df = generate_dataframe_world(full_table)
df.to_csv("covid.csv")


/var/folders/wh/st0xcv4923zgvl9dqddfcnf40000gn/T/ipykernel_4392/354785266.py:39: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['New_cases'] = df.New_cases.str.replace('+','')


In [166]:
Country = [row["Country"] for index, row in df.iterrows()]
Confirmed = [row["Confirmed"] for index, row in df.iterrows()]
New_cases = [row["New_cases"] for index, row in df.iterrows()]
Deaths = [row["Deaths"] for index, row in df.iterrows()]
Recovered = [row["Recovered"] for index, row in df.iterrows()]

In [167]:
revised_df = pd.DataFrame(
    {
        "Country" :Country,
        "Confirmed" :Confirmed,
        "New_cases" :New_cases,
        "Deaths" :Deaths,
        "Recovered" :Recovered,
    }
)

In [168]:
# Mongodb connection using Pymongo

client = MongoClient("mongodb://localhost/covid")
db = client.covid
covid_collection = db.covid_coviddata

In [169]:
# Inserting into Mongodb

for index, row in df.iterrows():
    covid_collection.insert_one(
        {
            "Id" : index,
            "Country" : row["Country"],
            "Confirmed" : row["Confirmed"],
            "New_cases" : row["New_cases"],
            "Deaths" : row["Deaths"],
            "Recovered " : row["Recovered"],
        }
    )
    

DuplicateKeyError: E11000 duplicate key error collection: covid.covid_coviddata index: __primary_key__ dup key: { Id: 0 }, full error: {'index': 0, 'code': 11000, 'keyPattern': {'Id': 1}, 'keyValue': {'Id': 0}, 'errmsg': 'E11000 duplicate key error collection: covid.covid_coviddata index: __primary_key__ dup key: { Id: 0 }'}